In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# define constants
IMG_SIZE = 100

In [3]:
def conv2d_out_sz(in_size, kernel_size, pool_size, padding=0, stride=1):
    return ((in_size - kernel_size + 2*padding)/stride + 1)/pool_size

In [4]:
ims = IMG_SIZE # image size
ks = 5 # kernel size
ps = 2 # pool size
out_chan_3 = 128 # out channels for conv 3

os1 = conv2d_out_sz(ims,ks,ps) # outputs to the next layer of conv2d
os2 = conv2d_out_sz(os1,ks,ps) # output size 2
os3 = conv2d_out_sz(os2,ks,ps) # output size 3
fcin = int((os3**2)*out_chan_3)

print(fcin)

10368


In [5]:
class WildfireModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.kernel_size = 5
        
        self.conv1 = nn.Conv2d(in_channels=3,  out_channels=32,  kernel_size=self.kernel_size)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,  kernel_size=self.kernel_size)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=self.kernel_size)

        self.fc1 = nn.Linear(fcin, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        x = x.view(-1, fcin)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
                    
        return x

In [6]:
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

In [7]:
data = np.load("data.npy", allow_pickle=True)

In [8]:
x = torch.tensor([d[0] for d in data])
x=(x/255.0).view(-1,3,100,100)

y = torch.tensor([float(d[1]) for d in data])

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.1)
(train_x.shape, train_y.shape), (test_x.shape, test_y.shape)

((torch.Size([890, 3, 100, 100]), torch.Size([890])),
 (torch.Size([99, 3, 100, 100]), torch.Size([99])))

In [11]:
wfm = WildfireModel()
optimizer = optim.SGD(wfm.parameters(), lr=0.01, momentum=0.9)
loss_function = nn.BCEWithLogitsLoss()

BATCH_SIZE = 100
EPOCHS = 7

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
        batch_x = train_x[i : i+BATCH_SIZE]
        batch_y = train_y[i : i+BATCH_SIZE]
        batch_y = torch.unsqueeze(batch_y, 1)
        
        wfm.zero_grad()

        outputs = wfm(batch_x)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.6535011529922485


  0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.6905972361564636


  0%|          | 0/9 [00:00<?, ?it/s]

...

Epoch: 4. Loss: 0.6007044911384583


  0%|          | 0/9 [00:00<?, ?it/s]

Epoch: 5. Loss: 0.5589374899864197


100%|██████████| 9/9 [00:17<00:00,  1.94s/it]

Epoch: 6. Loss: 0.5216952562332153


In [12]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_x))):
        real_class = test_y[i]
        net_out = wfm(test_x[i:i+1])[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)
        
        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

100%|██████████| 99/99 [00:00<00:00, 114.70it/s]

Accuracy:  0.737
